# Cifra Exponencial
<ul>
<li>Criptografia mais segura do que a criptografia afim </li>
<li>Trabalha com a codificação de blocos, não com codificação letra-a-letra </li>
<li>O bloco é primeiro codificado, em seguida é encriptado.</li>
</ul>


### Codificando um bloco:
Para criptografar um bloco, primeiro separamos um tamanho $N$ para os blocos. Em seguida, dado um número $B$ de caracteres, é realizada a seguinte conta:

$$
[c_1 c_2 ... c_N-1] = \sum_{i = 0}^{N - 1} C_i * B^i = K_x
$$

É notável que a codificação é inversível, portanto pode se obter o bloco através de $K_x$.


### Criptografando um Bloco:

* Tendo o texto separado em blocos:
$
\overbrace{[c_1c_2...c_N]}^{1^o \ bloco}
\overbrace{[c_{N+1}c_{N+2}...c_{2N}]}^{2^o \ bloco} ...
$
* O n-ésimo bloco é codificado com $K_n \in \mathbb{Z}$
* Seja um primo $P > K_n$ 
* Seja $ e \in \left\{ 2, ..., P - 1 \right\} $ um número _inversível_
  * $e$ é a chave da criptografia
* Seja $ d \in \left\{2, ..., P-1 \right\}$ o inverso de $e \ mod \ P-1$
  * $d$ será chave de descriptografia.
* A mensagem será um número $a \in \left\{0, ... P-1 \right\}$
* A mensagem será _criptografada_ como:
$$
b = C(a) = a^e \ mod \ P
$$
* A mensagem será _descriptografada_ como:
$$
D(a) = b^d \ mod \ P
$$
* Note que
$$
D(C(a)) = a\\
e * d = K (P-1) + 1
$$



### Codificando os números:
* À seguir, criamos funções para codificar e descodificar um bloco, de acordo com o número do mesmo

In [1]:

def codifica_bloco(mesg:str, size=20) -> list:
  """Função para codificar uma mensagem de acordo com o tamanho do Bloco"""
  
  bloco, n =  0, 0
  lista = []
  l_final = []
  
  while(n + size <= len(mesg)):
    lista.append(mesg[n:n + size])
    n += size
    
  if len(mesg)%size != 0:
    blocos_criados = (len(mesg) // size)
    lista.append(mesg[(blocos_criados * size):])

  for i in lista:
    mes = i
    
    for idx, s in enumerate(mes):
      bloco += ord(s) * 256**idx 
      
    l_final.append(bloco)
    bloco = 0
    
  return l_final


def decodifica_bloco(num):
  """Função para descodificar uma mensagem em blocos."""
  b = []

  for i in num:
    a = []
    c = []
    
    while i > 0:
      a.append(i%256)
      i = i//256

    
    for j in a:
      c.append(chr(j))
    b.append(str(c).strip().replace(',', '').replace('[', '').replace(']', ' ').replace("'", ''))
    
  return b

In [2]:
#Altere aqui a mensagem e o número de blocos que deseja em cada mensagem

mensagem = 'Hello World! Algebra?'
pre_cod = codifica_bloco(mensagem, 5)
decod = decodifica_bloco(pre_cod)

print(f'Pré-Codificação: {pre_cod}')
print(f'Decodificação: {decod} ')

Pré-Codificação: [478560413000, 465776367392, 464949092708, 418530878823, 63]
Decodificação: ['H e l l o ', '  W o r l ', 'd !   A l ', 'g e b r a ', '? '] 


### Criptografando a mensagem
Evidentemente que a mensagem codificada **não é** uma mensagem criptografada. À seguir, iremos criptografar as mensagens por meio da criptografia exponencial, cuja teoria está explicitada acima.

In [3]:
# Aqui serão definidas funções

def XMDC(a, b):
  """Algoritmo Extendido de Euclides, para cálculo das equações modulares"""
  x0, x = 1, 0
  y0, y = 0, 1
  
  sign_a, sign_b = 1,  1
  if a < 0:
    a = -a
    sign_a = -1
  
  if b < 0:
    b = -b
    sign_b = -1
    
  while b > 0:
    q, r = a//b, a%b
    a, b = b, r
    
    x, x0 = x0 - q * x, x
    y, y0 = y0 - q * y, y
    
  return a, sign_a * x0, sign_b * y0

def ExpModN(a, e, n):
  """Calcula a^e mod n"""
  A, P, E = a, 1, e
  while E != 0:
    D = E % 2
    
    if D == 1:
      P = (A*P) % n
      E = (E-1)//2
      
    else:
      E = E//2
      
    A = (A*A) % n
    
  return P

def inverso_modular(a, n):
  """Calcula o inverso modular de a em Zn"""
  d, u, v = XMDC(a, n)
  
  if d !=1: 
    return False
  
  u %= n
  
  if u < 0: 
    u += n
    
  return u

# Função para criptografar com o número e
cripto_exp = lambda n, p, e: ExpModN(n, e, p)

# Função para descriptografar com número d
decripto_exp = lambda n, p, d: ExpModN(n, d, p)

In [ ]:
# Para criptografarmos, utilizaremos um primo p = 2**607 - 1 (primo grande encontrado na wikipédia)
from random import randint

p = 2 ** 607 -1

d = False
e = 0

while not(d):
  e = randint(1, p-1)
  d = inverso_modular(e, p-1)

# Insira aqui a mensagem que deseja ciptografar
mensagem = ""
codificado = codifica_bloco(mensagem)

cripto = []

for i in codificado:
  cripto.append(cripto_exp(i, p, e))
  
decripto = []

for i in cripto:
  decripto.append(decripto_exp(i, p, d))
  
decod = decodifica_bloco(decripto)

print(f"""Mensagem Original: {mensagem} \n
Mensagem Codificada: {codificado} \n
Mensagem Criptografada: {cripto} \n
Mensagem Descriptografada: {decripto} \n
Mensagem Decodificada: {decod}""")

## Resumo Geral
A criptografia funciona de modo a permitir que apenas os indivíduos que possuam as chaves de criptografia $e$ e descriptografia $d$ tenham acesso à mensagem. 
Supondo dois indivíduos Alice e Bob, a troca de mensagens entre eles poderia ser representada por:
$$
\overbrace{Alice}^{Possui\ chave\ e} \to\ {mensagem \ é \ codificada}\ \to\ {mensagem\ é\ criptografada} \to\ \overbrace{Bob}^{Possui\ chave\ d} \to\ {Mensagem\ é\ descriptografada} \to\ {Mensagem \ é \ decodificada}
$$
